In [1]:
from operator import index
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
# import datetime
from datetime import datetime, timedelta
import re
import pandas as pd
import numpy as np


def to_int(text):
    import re
    p = re.compile(r'[ㄱ-힣,]+')
    return int(re.sub(p,"",text))

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

def for_you_kids(file_name, col_name):
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
    options.add_argument('headless')

    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

    df = pd.read_excel(file_name)
    product_list = df[col_name].to_list()

    img_url = []
    detail_img_url = []

    url = 'http://www.4ukids.co.kr/front/product_list'

    for i in product_list:
        time.sleep(2)
        driver.get(url = url)

        driver.find_element_by_css_selector('div.sch > form > input').send_keys(i)
        driver.find_element_by_css_selector('div.sch > form > button').click()

        time.sleep(2)
        try:
            driver.find_element_by_css_selector('span.name').click()
        except:
            pass

        try:
            element1 = driver.find_element_by_css_selector('#subview > div > div.detailbox > div.view_img > div.bx-wrapper > div.bx-viewport > ul > li:nth-child(2) > img')
            img = element1.get_attribute("src")
            img_url.append(img)
        except:
            img_url.append(np.nan)
        
        try:
            element2 = driver.find_element_by_css_selector('div.box > p > img')
            detail = element2.get_attribute("src")
            detail_img_url.append(detail)
        except:
            detail_img_url.append(np.nan)

    driver.quit()


    data = pd.DataFrame(product_list, columns=['product_list'])
    data['img_url'] = img_url
    data['detail_img_url'] = detail_img_url

    price_1 = []
    price_2 = []
    price_3 = []
    price_4 = []
    price_5 = []
    for i in product_list:
        url_lowest = 'https://search.shopping.naver.com/search/all?query='+str(i)+'&cat_id=&frm=NVSHATC'
        headers = {'users':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'}
        time.sleep(2)

        response = requests.get(url_lowest,headers=headers)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        try:
            p1 = to_int(soup.select('span.price_num__2WUXn')[0].get_text())
            price_1.append(p1)
        except:
            price_1.append(np.nan)

        try:
            p2 = to_int(soup.select('span.price_num__2WUXn')[1].get_text())
            price_2.append(p2)
        except:
            price_2.append(np.nan)

        try:
            p3 = to_int(soup.select('span.price_num__2WUXn')[2].get_text())
            price_3.append(p3)
        except:
            price_3.append(np.nan)

        try:
            p4 = to_int(soup.select('span.price_num__2WUXn')[3].get_text())
            price_4.append(p4)
        except:
            price_4.append(np.nan)
        
        try:
            p5 = to_int(soup.select('span.price_num__2WUXn')[4].get_text())
            price_5.append(p5)
        except:
            price_5.append(np.nan)

    data['price_1'] = price_1
    data['price_2'] = price_2
    data['price_3'] = price_3
    data['price_4'] = price_4
    data['price_5'] = price_5

    return data.to_excel("4ukids.xlsx",index=False)

In [ ]:
# 사용법: ecommerce('파일이름 + 확장자', '열 이름')
for_you_kids('재고현황 220218(물류팀)_잼토이즈_220224회신 copy.xlsx','포유키즈 상품정보')